In [1]:
#Load packages
import numpy as np
import os
import scipy.io as sio
from scipy.special import gammaln, softmax  
from scipy.linalg import solve_triangular, solve, eigh, eig
from itertools import groupby
import pandas as pd
import random
import scipy
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from numpy.linalg import matrix_rank
from scipy.stats import multivariate_t
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering

In [2]:
#Load data
whole = sio.loadmat('whole_wolabels.mat')
parts = sio.loadmat('parts_wolabels.mat')

In [3]:
#functions
def encode_labels(labels):
    le = LabelEncoder()
    le.fit(labels)
    encoded_labels = le.transform(labels)
    
    return encoded_labels, le

def decode_labels(encoded_predict_labels, le):
    test_predictions = le.inverse_transform(encoded_predict_labels)
    
    return test_predictions

def Harmonic_Mean(ytest, ypred, ytrain):
    ytest = np.array(ytest)
    ypred = np.array(ypred)
    true_dict = Counter(ytest)
    prediction_dict = Counter(ypred)
    uytest = set(ytest) #all classes present during testing may contain unseen classes 
                        # and only a subset of the seen classes
    uytrain = set(ytrain) #classes represented during training, seen classes
    
    #Mean class accuracy
    uy = list(uytest.intersection(uytrain))
    nc = len(uy)
    Classification_accuracy = np.zeros(nc)
    for i in range(nc):
        Index_of_i_th_class_true = list(np.argwhere(ytest == uy[i]).ravel())
        Index_of_i_th_class_pred = list(np.argwhere(ypred == uy[i]).ravel())
        tp = len(list(set(Index_of_i_th_class_true).intersection(Index_of_i_th_class_pred)))
        Classification_accuracy[i] = tp/true_dict[uy[i]]
    Classification_accuracy = np.mean(Classification_accuracy)
    
    #Out of distribution f1 score
    uy = set(uy)
    u_unseen_classes = list(uytest - uy)
    Index_of_unseen_class_true = list(np.argwhere(np.isin(ytest, u_unseen_classes)).ravel())
    Index_of_unseen_class_pred = list(np.argwhere(ypred > 5000).ravel())
    tp = len(list(set(Index_of_unseen_class_true).intersection(Index_of_unseen_class_pred)))
    fp = len(Index_of_unseen_class_pred) - tp
    fn = len(Index_of_unseen_class_true) - tp
    OOD_F1 = (2*tp)/(2*tp + fp + fn)
    
    harmonic_mean = 2*Classification_accuracy*OOD_F1/(Classification_accuracy+OOD_F1) 
                                    #harmonic mean of the two scores
    
    return Classification_accuracy, OOD_F1, harmonic_mean 

### Calculating class mean and covariance priors ###
def calculate_priors(xtrain, ytrain):
    d = xtrain.shape[1]
    uy = np.unique(ytrain)
    nc = len(uy)
    mu0 = np.zeros((1,d))
    Sigma0 = np.zeros((d,d))
    for i in range(nc):
        idx = (ytrain==uy[i]).flatten()
        if len(xtrain[idx,:]) == 1:
            mu0 += xtrain[idx,:] 
        else:
            mu0 = mu0 + np.mean(xtrain[idx,:], axis=0)
            Sigma0 = Sigma0 + np.cov(xtrain[idx,:], rowvar=False)
    mu0 = mu0/nc
    Sigma0 = Sigma0/nc
    
    return np.squeeze(mu0), Sigma0
    
### Calculating Posterior Predictive Distribution parameters ###
def calculate_ppd_params(xtrain, ytrain, unseenclasses, Psi, mu0, m, k0, k1):

    cnt = 0
    seenclasses = np.unique(ytrain)
    nc   = len(seenclasses) + len(unseenclasses)
    n, d = xtrain.shape

    Sig_s    = np.zeros((d,d,nc))
    Sigmas   = np.zeros((d,d,nc))
    mu_s     = np.zeros((nc, d))
    v_s      = np.zeros((nc, 1), dtype=np.int32)
    class_id = np.zeros((nc, 1))
    
    # The first part: for Seen classes
    uy              = seenclasses
    ncl             = len(uy)

    for i in range(ncl):
        idx         = np.in1d(ytrain, uy[i])
        Xi          = xtrain[idx]

        # The current selected component stats: # points, mean and scatter
        cur_n       = np.sum(idx)
        #print(cur_n)
        cur_S       = (cur_n-1)*np.cov(Xi.T)
        #print(cur_S)
        cur_mu      = np.mean(Xi, axis=0, keepdims=True)
        
        # The case where only data likelihood and global priors are used and local priors are ignored. This 
        # is the case we used for seen classes as mentioned in the paper
        v_s[cnt]        = cur_n+m-d+1
        mu_s[cnt]       = (cur_n*cur_mu+(k0*k1/(k0+k1))*mu0)/(cur_n+(k0*k1/(k0+k1)))
        Smu             = ((cur_n*(k0*k1/(k0+k1)))/((k0*k1/(k0+k1))+cur_n))*np.dot(cur_mu-mu0, (cur_mu-mu0).T)
        if cur_n == 1:
            Sig_s[:,:,cnt]  = (Psi+Smu)/(((cur_n+(k0*k1/(k0+k1)))*v_s[cnt])/(cur_n+(k0*k1/(k0+k1))+1))
        else:
            Sig_s[:,:,cnt]  = (Psi+cur_S+Smu)/(((cur_n+(k0*k1/(k0+k1)))*v_s[cnt])/(cur_n+(k0*k1/(k0+k1))+1))
        if np.isnan(np.sum(Sig_s[:,:,i])):
            print(i, "th cov is nan")
            
        class_id[cnt]   = uy[i]
        cnt            += 1

    # The second part: creating local priors for each Genus
    ncl           = len(unseenclasses)
    
    # Main for loop for  Genus local priors params estimation
    for i in range(ncl):
        classes = unseenclasses[i]
        #Extract corresponding data
        idx       = np.in1d(ytrain, classes)
        Yi        = ytrain[idx]
        Xi        = xtrain[idx]
        uyi       = np.unique(Yi)

        # Initialize component sufficient statistics 
        ncpi      = len(uyi)
        xkl       = np.zeros((ncpi,d))      # Component means
        Skl       = np.zeros((d,d,ncpi))    # Component scatter matrices
        kap       = np.zeros((ncpi,1))      # model specific
        nkl       = np.zeros((ncpi,1))      # number of data points in the components

        # Calculate  sufficient statistics for each component in meta cluster
        for j in range(ncpi):
            idx        = np.in1d(Yi, uyi[j])
            nkl[j]     = np.sum(idx)
            kap[j]     = nkl[j]*k1/(nkl[j]+k1)
            Xij        = Xi[idx]
            xkl[j]     = np.mean(Xij, axis=0)
            if nkl[j] > 1:
                Skl[:,:,j] = (nkl[j]-1)*np.cov(Xij.T)   

        # Model specific parameters
        sumkap       = np.sum(kap)
        kaps         = (sumkap+k0)*k1/(sumkap+k0+k1)
        sumSkl       = np.sum(Skl,axis=2)
        muk          = (np.sum(np.multiply(xkl, kap*np.ones((1,d))), axis=0)+k0*mu0)/(sumkap+k0)
        
        # Unseen classes' predictive cov, mean and dof
        v_s[cnt]         = np.sum(nkl)-ncpi+m-d+1
        class_id[cnt]   = 5000 + cnt
        Sigmas[:,:,cnt] = Psi+sumSkl
        Sig_s[:,:,cnt]  = (Psi+sumSkl)/((kaps*v_s[cnt])/(kaps+1))
        if np.isnan(np.sum(Sig_s[:,:,cnt])):
            print(cnt, "th cov is nan")
            print(Psi)
            print(sumSkl)
        mu_s[cnt]       = muk
        cnt             += 1  

    return Sig_s, mu_s, v_s, class_id, Sigmas 

### PPD calculation (Log-Likelihood of Student-t) ###
def bayesian_cls_evaluate(xtest, all_loc_vec, all_scale_mat, all_dof, all_labels):
    nr_classes = len(all_labels)
    log_lik = np.zeros((xtest.shape[0], nr_classes))
    for i in range(nr_classes):
        log_lik[:, i] = multivariate_t.logpdf(xtest, all_loc_vec[i], \
                                              all_scale_mat[:, :, i], all_dof[i])
    
    predicted_class_ids = np.argmax(log_lik, axis=1)
    ypred = np.zeros(len(xtest))
    
    for i in range(len(predicted_class_ids)):
        ypred[i] = all_labels[predicted_class_ids[i]]
    
    return ypred

def bayesian_cls_train(x_tr, y_tr, unseenclasses, k_0=0.1, k_1=10, m=5*500, mu_0=0, s=1):
    s_classes   = np.unique(y_tr)   
    d0          = x_tr.shape[1] 
    [mu_0, Sigma_0] = calculate_priors(x_tr, y_tr)
    #print("Sigma_0:", Sigma_0)
    Psi=(m-d0-1)*Sigma_0/s
    #print("Psi:", Psi)

    # Class predictive cov, mean and DoF from unconstrained model
    #print('PPD derivation is Done!!')
    return calculate_ppd_params(x_tr, y_tr, unseenclasses, Psi, mu_0, m, k_0, k_1) 

In [4]:
#whole

#train
train_classid = np.squeeze(whole['train_classid'])
train_class_labels = []
for item in train_classid:
    train_class_labels.append(item[0])
train_features = whole['train_feats']
train_imid = whole['train_imgid']
train_imgid = []
for item in train_imid:
    train_imgid.append(item[0])
train_imgid = np.squeeze(train_imgid)
train_sampleid = whole['train_sampleid']

#validation
validation_classid = np.squeeze(whole['val_classid'])
validation_class_labels = []
for item in validation_classid:
    validation_class_labels.append(item[0])
validation_features = whole['val_feats']
validation_imid = whole['val_imgid']
validation_imgid = []
for item in validation_imid:
    validation_imgid.append(item[0])
validation_imgid = np.squeeze(validation_imgid)
validation_sampleid = whole['val_sampleid']

#test
test_features = whole['test_feats']
test_imid = whole['test_imgid']
test_imgid = []
for item in test_imid:
    test_imgid.append(item[0])
test_imgid = np.squeeze(test_imgid)
test_sampleid = whole['test_sampleid']

#encoded train labels
train_labels, le = encode_labels(train_class_labels)
train_unique_labels = sorted(np.unique(train_labels))
train_unique_labels_count = len(train_unique_labels)
print(len(train_labels))

#encoded validation labels
validation_labels = le.transform(validation_class_labels)
validation_unique_labels = sorted(np.unique(validation_labels))
validation_unique_labels_count = len(validation_unique_labels)
print(len(validation_labels))

7849
1379


In [5]:
#parts

#train
train_classid_parts = np.squeeze(parts['train_classid'])
train_class_labels_parts = []
for item in train_classid_parts:
    train_class_labels_parts.append(item[0])
train_features_parts = parts['train_feats']
train_imid_parts = parts['train_imgid']
train_imgid_parts = []
for item in train_imid_parts:
    train_imgid_parts.append(item[0])
train_imgid_parts = np.squeeze(train_imgid_parts)
train_sampleid_parts = parts['train_sampleid']
train_tileid_parts = parts['train_tileid']

#validation
validation_classid_parts = np.squeeze(parts['val_classid'])
validation_class_labels_parts = []
for item in validation_classid_parts:
    validation_class_labels_parts.append(item[0])
validation_features_parts = parts['val_feats']
validation_imid_parts = parts['val_imgid']
validation_imgid_parts = []
for item in validation_imid_parts:
    validation_imgid_parts.append(item[0])
validation_imgid_parts = np.squeeze(validation_imgid_parts)
validation_sampleid_parts = parts['val_sampleid']
validation_tileid_parts = parts['val_tileid']

#test
test_features_parts = parts['test_feats']
test_imid_parts = parts['test_imgid']
test_imgid_parts = []
for item in test_imid_parts:
    test_imgid_parts.append(item[0])
test_imgid_parts = np.squeeze(test_imgid_parts)
test_sampleid_parts = parts['test_sampleid']
test_tileid_parts = parts['test_tileid']

In [6]:
train_n, d = train_features_parts.shape
print(train_n,d)
train_features_parts_bags = train_features_parts.reshape(int(train_n/9), 9, d)
print(train_features_parts_bags.shape)

val_n, d = validation_features_parts.shape
print(val_n,d)
validation_features_parts_bags = validation_features_parts.reshape(int(val_n/9), 9, d)
print(validation_features_parts_bags.shape)

70641 384
(7849, 9, 384)
12411 384
(1379, 9, 384)


In [7]:
weight = [0.1165491,  0.10607711, 0.11824138, 0.12487643, 0.1178225,  0.12241342, 0.10193858, 0.0929913,  0.09909019]

combined_train_features = []

for i in range(len(train_features_parts_bags)):
    train_features_parts_weighted_mean = np.zeros(d)
    for j in range(len(train_features_parts_bags[i])):
        train_features_parts_weighted_mean += train_features_parts_bags[i][j] * weight[j]
    combined_train_features.append(np.concatenate((train_features[i], train_features_parts_weighted_mean), axis=None))

print(np.array(combined_train_features).shape)

combined_validation_features = []
for i in range(len(validation_features_parts_bags)):
    validation_features_parts_weighted_mean = np.zeros(d)
    for j in range(len(validation_features_parts_bags[i])):
        validation_features_parts_weighted_mean += validation_features_parts_bags[i][j] * weight[j]
    combined_validation_features.append(np.concatenate((validation_features[i], validation_features_parts_weighted_mean), axis=None))

print(np.array(combined_validation_features).shape)

(7849, 768)
(1379, 768)


Minmax normalization

In [8]:
scaler = MinMaxScaler()
train_features_norm = scaler.fit_transform(combined_train_features)
validation_features_norm = scaler.transform(combined_validation_features)

Creating holdout dataset

In [9]:
# combine train and validation data labels
total_labels = np.array(list(train_labels) + list(validation_labels))
print(len(total_labels))

validation_count_dict = Counter(validation_labels)
#print(validation_count_dict)

validation_labels_dict = sorted(validation_count_dict.items(), key=lambda x: x[1], reverse=True)
unique_unseen_labels = []
for i in range(75,125):
      unique_unseen_labels.append(validation_labels_dict[i][0])
print("unique_unseen_labels:", unique_unseen_labels)

unseen_labels_index = np.argwhere(np.isin(total_labels, unique_unseen_labels)).ravel()
print("unseen_labels length:", len(unseen_labels_index))

#separating unseen classes from train data
unseen_labels_index_train = np.argwhere(np.isin(train_labels, unique_unseen_labels)).ravel()
unseen_labels_train = train_labels[unseen_labels_index_train]
unseen_embeddings_train = train_features_norm[unseen_labels_index_train]

#separating seen classes from train data
unique_seen_labels = sorted(list(set(train_unique_labels) - set(unique_unseen_labels)))
print("unique_seen_labels length:", len(unique_seen_labels))

seen_labels_index_train = np.argwhere(np.isin(train_labels, unique_seen_labels)).ravel()
print("seen_labels index train length:", len(seen_labels_index_train))
seen_labels_train = train_labels[seen_labels_index_train].reshape(len(seen_labels_index_train)).tolist()
seen_embeddings_train = train_features_norm[seen_labels_index_train]

#Final train data
xtrain = seen_embeddings_train
ytrain = np.array(seen_labels_train)

#final validation data
xtest = np.vstack((validation_features_norm, unseen_embeddings_train))
ytest = np.concatenate((validation_labels, unseen_labels_train))

#print(train_embeddings)
print(xtrain.shape)
#print(train_labels)
print(len(ytrain))
#print(validation_embeddings)
print(xtest.shape)
#print(validation_labels)
print(len(ytest))

9228
unique_unseen_labels: [49, 51, 60, 61, 78, 84, 86, 88, 89, 100, 103, 106, 107, 110, 111, 113, 116, 121, 132, 134, 145, 146, 148, 152, 154, 158, 168, 175, 177, 199, 205, 207, 209, 212, 221, 222, 225, 227, 232, 236, 241, 249, 251, 255, 256, 257, 272, 273, 275, 281]
unseen_labels length: 706
unique_seen_labels length: 963
seen_labels index train length: 7243
(7243, 768)
7243
(1985, 768)
1985


Steps:
    
1. Calculate centroids of each classes

2. Pass these centroids into an agglomerative clustering algorithm

3. After getting class ids group classes to feed in student-t as usclass.

In [10]:
#Calculate centroids of each classes

d = xtrain.shape[1]
uy = np.unique(ytrain)
nc = len(uy)
mu_centroids = np.zeros((nc,d))
for i in range(nc):
    idx = (ytrain==uy[i]).flatten()
    mu_centroids[i] = np.mean(xtrain[idx,:], axis=0)

print(mu_centroids)
print(mu_centroids.shape)

[[0.65556688 0.5141114  0.51305535 ... 0.62419207 0.54914402 0.4438058 ]
 [0.19557025 0.40848557 0.37175681 ... 0.57214728 0.41011728 0.50032743]
 [0.49969004 0.56387997 0.39936086 ... 0.70321405 0.37981569 0.50932528]
 ...
 [0.54548229 0.52888306 0.48156724 ... 0.45522464 0.5675549  0.3842312 ]
 [0.63067382 0.46906015 0.40887614 ... 0.6734135  0.32067259 0.55549043]
 [0.42478056 0.69123404 0.33646051 ... 0.78979142 0.55377144 0.62955589]]
(963, 768)


Agglomerative clustering

n_clusters = 100, distance=ward

In [11]:
# Pass these centroids into an agglomerative clustering algorithm
n_clusters = 100
clustering = AgglomerativeClustering(n_clusters = n_clusters, affinity='euclidean', linkage='ward')
labels = clustering.fit_predict(mu_centroids)
print(labels)
print(np.unique(labels))

[ 7 14 32 53 35 91 39 35 35 80 80 23 35 35 58 17 17 58 80 72 58 80 35 58
  8 35 58 58 58 58 35 17 58 35 58 35 58 80 79 37 37 37 53 53 53 29 43 32
 35 62 61  3 93 26 18 18 18  5 96 17 17 96 93 51 51 40 46 80 80 47 10  7
 11 37 94 94 30 11 54 46 79 17 14 17 17 29 10 96 79 35 29 17 14 17 74  6
  6  6  6  6  7 36 53 75 75  2 39 11 48 18 17 17 17 85 53 53  3 81 81 81
 81  4 59 59 59 41 17 17 15 20 61 61 61 48 28 45 45 44 50 62 36 36 23 30
 30 30 23 83  0  0  0  0  0 88 88  0  0  0 83  0 83 88  0  0  0  0  0 83
  0 88 88 83 88  0  0  0 88  0  0 83 83 88  0 83  0  0 88  0  0 42  1 16
 85 72 49 49 81 32 92 92 23  3  3  3  3 87 87 87 31 31 31  7 31 31 31 16
 16 12 12 12 46 54 54 54 54 72  4 20 20 20 32 46 13 55 55 55 55 55 55 55
 55 72 55 55 55 55  8 55 55 55 55 50 52 25 20 44 44 10 10 10 10 10 50  3
 79 72 67 25 25 25 25 55 20 19 19 19 19 19 93 19 19 69 19 24 46  2 11 52
 11 47 47 56  8  8 35 17  8  8 35 35  8  8 91 10 10 50 26 25 25  3 53  8
 51 52 71 26 42 42 23 41 11 70 70 70 70 70 70 10 34

In [12]:
# After getting class ids group classes to feed in student-t as usclass.
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = [value]

cluster_dict = {}

for i in range(nc):
    append_value(cluster_dict, labels[i], uy[i])

print(cluster_dict)

{7: [0, 75, 117, 254, 406, 407, 408, 573, 744, 745, 814, 815, 816, 1009, 1010], 14: [1, 91, 102, 435, 450, 716, 717, 873, 984, 985, 986, 987, 988], 32: [2, 47, 237, 279, 405, 540, 643, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 725, 742, 867, 890, 961], 53: [3, 42, 43, 44, 119, 133, 135, 360, 634, 778, 1011], 35: [4, 7, 8, 12, 13, 22, 25, 30, 33, 35, 48, 98, 344, 348, 349, 423, 425, 444, 445, 446, 447, 448, 452, 456, 460, 462, 467, 468, 515, 576, 895, 913], 91: [5, 352, 629], 39: [6, 124, 755, 941], 80: [9, 10, 18, 21, 37, 71, 72, 770, 823], 23: [11, 169, 173, 240, 368, 753, 832, 878, 899, 955, 966], 58: [14, 17, 20, 23, 26, 27, 28, 29, 32, 34, 36], 17: [15, 16, 31, 63, 64, 90, 92, 93, 101, 104, 128, 129, 130, 149, 150, 345, 404, 430, 538, 599, 670, 673, 684, 771, 772, 799, 800, 820, 910, 962], 72: [19, 231, 271, 291, 315, 426, 593, 595, 596], 8: [24, 296, 342, 343, 346, 347, 350, 351, 361, 648, 758, 759, 760], 79: [38, 87, 97, 314, 421, 440, 718, 796, 818, 825, 874, 908, 977, 9

In [13]:
cluster_list = []
for x in cluster_dict:
    cluster_list.append(cluster_dict[x])
    
print(cluster_list)

[[0, 75, 117, 254, 406, 407, 408, 573, 744, 745, 814, 815, 816, 1009, 1010], [1, 91, 102, 435, 450, 716, 717, 873, 984, 985, 986, 987, 988], [2, 47, 237, 279, 405, 540, 643, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 725, 742, 867, 890, 961], [3, 42, 43, 44, 119, 133, 135, 360, 634, 778, 1011], [4, 7, 8, 12, 13, 22, 25, 30, 33, 35, 48, 98, 344, 348, 349, 423, 425, 444, 445, 446, 447, 448, 452, 456, 460, 462, 467, 468, 515, 576, 895, 913], [5, 352, 629], [6, 124, 755, 941], [9, 10, 18, 21, 37, 71, 72, 770, 823], [11, 169, 173, 240, 368, 753, 832, 878, 899, 955, 966], [14, 17, 20, 23, 26, 27, 28, 29, 32, 34, 36], [15, 16, 31, 63, 64, 90, 92, 93, 101, 104, 128, 129, 130, 149, 150, 345, 404, 430, 538, 599, 670, 673, 684, 771, 772, 799, 800, 820, 910, 962], [19, 231, 271, 291, 315, 426, 593, 595, 596], [24, 296, 342, 343, 346, 347, 350, 351, 361, 648, 758, 759, 760], [38, 87, 97, 314, 421, 440, 718, 796, 818, 825, 874, 908, 977, 978, 979, 981], [39, 40, 41, 77, 834, 835, 836], [45, 9

In [14]:
cluster_dict_decoded = []
for x in cluster_dict:
    cluster_dict_decoded.append(decode_labels(cluster_dict[x], le)) 
    
print(cluster_dict_decoded)

[array(['Abaeis nicippe', 'Anteos clorinde', 'Aphrissa statira',
       'Colias interior', 'Eurema daira', 'Eurema mexicana',
       'Eurema salome', 'Kricogonia lyside', 'Phoebis agarithe',
       'Phoebis sennae', 'Pyrisitia dina', 'Pyrisitia lisa',
       'Pyrisitia proterpia', 'Zerene cesonia', 'Zerene eurydice'],
      dtype='<U28'), array(['Abagrotis bimarginalis', 'Apamea cogitata', 'Apamea scoparia',
       'Euxoa lewisi', 'Euxoa perpolita', 'Parabagrotis exsertistigma',
       'Parabagrotis formalis', 'Sideridis maryx', 'Xylena brucei',
       'Xylena cineritia', 'Xylena curvimacula', 'Xylena nupera',
       'Xylena thoracica'], dtype='<U28'), array(['Acerra normalis', 'Agrochola purpurea', 'Cissusa valens',
       'Crocigrapha normani', 'Eupsilia tristigmata',
       'Homoglaea carbonaria', 'Mesogona olivata',
       'Orthosia annulimacula', 'Orthosia behrensiana',
       'Orthosia erythrolita', 'Orthosia hibisci', 'Orthosia mys',
       'Orthosia pacifica', 'Orthosia praeses

In [15]:
count = 0
for i in range(len(cluster_dict_decoded)):
    print(len(cluster_dict_decoded[i]))
    count += len(cluster_dict_decoded[i])
print(len(cluster_dict_decoded))
print(count)

15
13
22
11
32
3
4
9
11
11
30
9
13
16
7
24
7
9
17
17
16
14
19
5
10
16
7
20
5
19
18
6
16
14
11
13
6
3
9
4
5
7
10
3
7
14
17
27
4
4
18
8
26
9
7
11
15
5
2
3
7
7
7
20
10
23
3
16
6
9
6
1
6
4
4
4
3
7
7
4
1
3
3
12
3
4
5
7
5
2
3
6
6
6
3
7
5
1
5
9
100
963


Train and evaluation: hyper parameter tuning

In [17]:
d = xtrain.shape[1]

all_kappa_0s = [0.1, 1, 10]
all_kappa_1s = [0.01, 0.1, 1, 10]
all_ms = [d+2, 100*d]
all_s = [1, 10]

for k_0 in all_kappa_0s:
    for k_1 in all_kappa_1s:
        for m in all_ms:
            for s in all_s:
                ### PPD parameter estimation ###
                Sig_s, mu_s, v_s, class_id, _= bayesian_cls_train(xtrain, ytrain, cluster_list, k_0=k_0, k_1=k_1, m=m, s=s)

                ### Prediction phase ###
                ypred = bayesian_cls_evaluate(xtest, mu_s, Sig_s, v_s, class_id) 

                Classification_accuracy, OOD_F1, harmonic_mean = Harmonic_Mean(ytest, ypred, ytrain)

                print('Results from k0=%.2f, k1=%.2f, m=%d, s=%.1f:' % (k_0, k_1, m, s))
                print('Mean class acc: %.2f OOD F1: %.2f, Harmonic mean: %.2f'%(Classification_accuracy, OOD_F1, harmonic_mean))

/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/ipykernel_launcher.py:93: RuntimeWarning: Degrees of freedom <= 0 for slice
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


Results from k0=0.10, k1=0.01, m=770, s=1.0:
Mean class acc: 0.82 OOD F1: 0.33, Harmonic mean: 0.47
Results from k0=0.10, k1=0.01, m=770, s=10.0:
Mean class acc: 0.78 OOD F1: 0.48, Harmonic mean: 0.59
Results from k0=0.10, k1=0.01, m=76800, s=1.0:
Mean class acc: 0.83 OOD F1: 0.04, Harmonic mean: 0.07
Results from k0=0.10, k1=0.01, m=76800, s=10.0:
Mean class acc: 0.00 OOD F1: 0.53, Harmonic mean: 0.01
Results from k0=0.10, k1=0.10, m=770, s=1.0:
Mean class acc: 0.83 OOD F1: 0.21, Harmonic mean: 0.34
Results from k0=0.10, k1=0.10, m=770, s=10.0:
Mean class acc: 0.83 OOD F1: 0.12, Harmonic mean: 0.21
Results from k0=0.10, k1=0.10, m=76800, s=1.0:
Mean class acc: 0.81 OOD F1: 0.22, Harmonic mean: 0.35
Results from k0=0.10, k1=0.10, m=76800, s=10.0:
Mean class acc: 0.00 OOD F1: 0.53, Harmonic mean: 0.01
Results from k0=0.10, k1=1.00, m=770, s=1.0:
Mean class acc: 0.82 OOD F1: 0.11, Harmonic mean: 0.20
Results from k0=0.10, k1=1.00, m=770, s=10.0:
Mean class acc: 0.83 OOD F1: 0.07, Harmoni

KeyboardInterrupt: 